<a href="https://colab.research.google.com/github/vernikagupta/Deep_Learning_with_Maths/blob/master/Keras_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
IMAGE_SIZE = [512, 512]

In [ ]:
#Give dataset path
# path = '/content/drive/MyDrive/Work/CNN architechtures/CNN Architechtures/p&g data'
train_path = '/content/drive/MyDrive/Work/CNN architechtures/CNN Architechtures/p&g data/train'
valid_path =  '/content/drive/MyDrive/Work/CNN architechtures/CNN Architechtures/p&g data/val'
test_path = '/content/drive/MyDrive/Work/CNN architechtures/CNN Architechtures/p&g data/test'

In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# !unzip 'data (1).zip'

In [ ]:
# !ls

In [ ]:
# useful for getting number of classes
folders = glob('/content/drive/MyDrive/Work/CNN architechtures/CNN Architechtures/data/train/*')
print(len(folders))

2


In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(1, activation='sigmoid')(x)  
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)     0     

In [ ]:
from keras import optimizers


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy',"binary_accuracy"])

In [ ]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
valid_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [ ]:
# Data Augmentation
# test_datagen = ImageDataGenerator(
#     preprocessing_function=preprocess_input,
#     rotation_range=40,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

In [ ]:
# Make sure you provide the same target size as initialied for the image size
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Work/CNN architechtures/CNN Architechtures/data/train',
                                                 target_size = (512, 512),
                                                color_mode="rgb",
                                                 batch_size = 32,
                                                  shuffle=True,
                                                 seed=42,
                                                 class_mode = 'binary')

Found 79 images belonging to 2 classes.


In [ ]:
valid_set = valid_datagen.flow_from_directory(
    directory='/content/drive/MyDrive/Work/CNN architechtures/CNN Architechtures/data/val',
    target_size=(512, 512),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 116 images belonging to 2 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Work/CNN architechtures/CNN Architechtures/data/test',
                                            target_size = (512, 512),
                                            batch_size = 1,
                                            class_mode = 'binary')

NameError: ignored

In [ ]:
from datetime import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau

#lr_scheduler = LearningRateScheduler(lr_schedule)

#lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                               cooldown=0,
#                               patience=5,
#                               min_lr=0.5e-6)

#num_epochs = 1000
batch_size = 2

checkpoint = ModelCheckpoint(filepath='vgg16.h5', monitor='val_loss',
                               verbose=1, save_best_only=True,mode='auto')

callbacks = [checkpoint]

start = datetime.now()

hist = model.fit_generator(
  train_set,
  validation_data=valid_set,
  epochs=10,
  steps_per_epoch=len(train_set)//batch_size,
  validation_steps=len(valid_set)//batch_size ,
  callbacks=callbacks ,verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1/1 [==============================] - 12s 12s/step - loss: 1.2609 - accuracy: 0.6250 - binary_accuracy: 0.6250 - val_loss: 99.1937 - val_accuracy: 0.4286 - val_binary_accuracy: 0.4286

Epoch 00001: val_loss improved from inf to 99.19375, saving model to vgg16.h5
Epoch 2/10
1/1 [==============================] - 8s 8s/step - loss: 39.6429 - accuracy: 0.7500 - binary_accuracy: 0.7500 - val_loss: 11.1531 - val_accuracy: 0.8036 - val_binary_accuracy: 0.8036

Epoch 00002: val_loss improved from 99.19375 to 11.15313, saving model to vgg16.h5
Epoch 3/10
1/1 [==============================] - 8s 8s/step - loss: 40.9690 - accuracy: 0.4688 - binary_accuracy: 0.4688 - val_loss: 217.2173 - val_accuracy: 0.5357 - val_binary_accuracy: 0.5357

Epoch 00003: val_loss did not improve from 11.15313
Epoch 4/10
1/1 [==============================] - 7s 7s/step - loss: 48.9784 - accuracy: 0.8667 - binary_accuracy: 0.8667 - val_loss: 354.0854 - val_accuracy: 0.5000 - val_binary_accuracy: 0.5000

